# Image Classification 

# # # Import the required packages

In [53]:
import selenium 
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException
import requests
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string
import os
import urllib3
import urllib.request
#import requests
#from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

# Collecting the Saree images path and its titlefrom Amazon site

In [101]:
s_links = ['https://www.amazon.in/s?k=saree&ref=nb_sb_noss','https://www.amazon.in/s?k=saree&page=2&qid=1623300883&ref=sr_pg_2','https://www.amazon.in/s?k=saree&page=3&qid=1623304127&ref=sr_pg_3','https://www.amazon.in/s?k=saree&page=4&qid=1623304133&ref=sr_pg_4','https://www.amazon.in/s?k=saree&page=5&qid=1623331918&ref=sr_pg_5']
j_links = ['https://www.amazon.in/s?k=jeans+for+men+stylish&crid=1HP2F7L5NFFWR&sprefix=jeans%2Caps%2C312&ref=nb_sb_ss_ts-doa-p_1_5','https://www.amazon.in/s?k=jeans+for+men+stylish&page=2&crid=1HP2F7L5NFFWR&qid=1623304464&sprefix=jeans%2Caps%2C312&ref=sr_pg_2','https://www.amazon.in/s?k=jeans+for+men+stylish&page=3&crid=1HP2F7L5NFFWR&qid=1623304518&sprefix=jeans%2Caps%2C312&ref=sr_pg_3','https://www.amazon.in/s?k=jeans+for+men+stylish&page=4&crid=1HP2F7L5NFFWR&qid=1623304569&sprefix=jeans%2Caps%2C312&ref=sr_pg_4']
t_links = ['https://www.amazon.in/s?k=trousers+for+men&ref=nb_sb_noss_2','https://www.amazon.in/s?k=trousers+for+men&page=2&qid=1623304681&ref=sr_pg_2','https://www.amazon.in/s?k=trousers+for+men&page=3&qid=1623304700&ref=sr_pg_3','https://www.amazon.in/s?k=trousers+for+men&page=4&qid=1623304728&ref=sr_pg_4']
Saree_path = 'C:/Users/735184/OneDrive - Cognizant/Desktop/Digvijay/Image_Classification/Saree/'
Jeans_path = 'C:/Users/735184/OneDrive - Cognizant/Desktop/Digvijay/Image_Classification/Jeans/'
Trousers_path = 'C:/Users/735184/OneDrive - Cognizant/Desktop/Digvijay/Image_Classification/Trousers/'

img_url_jeans = []
img_url_jeans_title = []
img_url_trousers = []
img_url_trousers_title = []
img_url_saree = []
img_url_saree_title = []


def extract_images_amz (link_list,list_img,List_title):
    driver=webdriver.Chrome('C://Users//735184//Downloads//chromedriver_win32_New//chromedriver.exe')
    driver.maximize_window()
    time.sleep(1)
    for i in link_list:
        driver.get(i)
        image_cls = driver.find_elements_by_xpath("//img[@class='s-image']")
        image_title = driver.find_elements_by_xpath("//span[@class='a-size-base-plus a-color-base a-text-normal']")
        for i in image_cls:
            list_img.append(i.get_attribute('src'))
        for i in image_title:
            List_title.append(i.text)    


# Download the images at particular location on desktop

In [102]:
extract_images_amz(s_links,img_url_saree,img_url_saree_title)
time.sleep(5)
for i in range(len(img_url_saree[:241])):
    nm = 'Saree_'+str(i)+'.jpg'
    image = urllib.request.URLopener()
    image.retrieve(img_url_saree[i],Saree_path+nm)
time.sleep(15)
extract_images_amz(j_links,img_url_jeans,img_url_jeans_title)
time.sleep(5)
for i in range(len(img_url_jeans[:241])):
    nm = 'Jeans_'+str(i)+'.jpg'
    image = urllib.request.URLopener()
    image.retrieve(img_url_jeans[i],Jeans_path+nm)
time.sleep(15)
extract_images_amz(t_links,img_url_trousers,img_url_trousers_title)
for i in range(len(img_url_trousers[:241])):
    nm = 'Trousers_'+str(i)+'.jpg'
    image = urllib.request.URLopener()
    image.retrieve(img_url_trousers[i],Trousers_path+nm)



# Lets Start with the Image Classification 

In [106]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
  
img_width, img_height = 224, 224

Using TensorFlow backend.


In [115]:
train_image = 'C:/Users/735184/OneDrive - Cognizant/Desktop/Digvijay/Image_Class_Project/Image_Classification/'
validation_image = 'C:/Users/735184/OneDrive - Cognizant/Desktop/Digvijay/Image_Class_Project/Validation'

nb_train_samples =663
nb_validation_samples = 57
epochs = 10
batch_size = 16
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    

In [116]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
  
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
  
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
  
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [122]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
  
test_datagen = ImageDataGenerator(rescale=1. / 255)
  
train_generator = train_datagen.flow_from_directory(
    train_image,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
  
validation_generator = test_datagen.flow_from_directory(
    validation_image,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')
  
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Found 663 images belonging to 3 classes.
Found 57 images belonging to 3 classes.
Epoch 1/10
41/41 [==============================] - 19s 452ms/step - loss: -1.4124 - accuracy: 0.3338 - val_loss: -14.1838 - val_accuracy: 0.3125
Epoch 2/10
41/41 [==============================] - 20s 482ms/step - loss: -28.8241 - accuracy: 0.3431 - val_loss: 12.7645 - val_accuracy: 0.2927
Epoch 3/10
41/41 [==============================] - 20s 485ms/step - loss: -181.6719 - accuracy: 0.3338 - val_loss: -736.2914 - val_accuracy: 0.3659
Epoch 4/10
41/41 [==============================] - 22s 530ms/step - loss: -703.4667 - accuracy: 0.3308 - val_loss: 981.5819 - val_accuracy: 0.4146
Epoch 5/10
41/41 [==============================] - 22s 531ms/step - loss: -1044.7633 - accuracy: 0.3462 - val_loss: 616.8278 - val_accuracy: 0.3542
Epoch 6/10
41/41 [==============================] - 22s 547ms/step - loss: -2135.2156 - accuracy: 0.3570 - val_loss: -590.2169 - val_accuracy: 0.4634
Epoch 7/10
41/41 [=============

In [124]:
model.save_weights('model_saved.h5')